In [ ]:
import numpy as np
import csv

In [ ]:
tt = [200,500,1000,1500,2000] #K
sysname = "H_555567777+SV-3"
vibsfile = "vib_"+sysname+".txt"
H_infiles = ["ChemPotH_200K.txt", "ChemPotH_500K.txt", "ChemPotH_1000K.txt", "ChemPotH_1500K.txt", "ChemPotH_2000K.txt"]
O_infiles = ["ChemPotO_200K.txt", "ChemPotO_500K.txt", "ChemPotO_1000K.txt", "ChemPotO_1500K.txt", "ChemPotO_2000K.txt"]
outfiles = ["./200K/"+sysname+"_200K.txt", "./500K/"+sysname+"_500K.txt", "./1000K/"+sysname+"_1000K.txt", 
            "./1500K/"+sysname+"_1500K.txt", "./2000K/"+sysname+"_2000K.txt"]
counter = 0
for T in tt:
    #define constants
    size = 200
    vib_remove = 6
    kB = 8.6173303e-5 #eV/K
    pi = np.pi
    P = np.linspace(-12, 5, size) 
    P = 10**P #bar
    P = P*10**5 #Pa

    #define energies, USER INPUTS FROM DFT MODELS
    Eform  = 8.255715458 #eV
    N_vac = 3
    N_O = 0
    N_H = 3

    #import chemical potentials
    CP_Ho = [] #hydrogen
    with open(H_infiles[counter]) as f:    
        for line in f:
            CP_Ho.append(line)

    CP_H = []
    for ii in range(0,size,1):
        test = CP_Ho[ii].split()
        map_object = map(float, test)
        temp = list(map_object)
        for jj in range(0,size,1):
            CP_H.append(temp[jj])

    CP_H = np.reshape(CP_H,(size,size))

    CP_Oo = [] #oxygen
    with open(O_infiles[counter]) as f:    
        for line in f:
            CP_Oo.append(line)

    CP_O = []
    for ii in range(0,size,1):
        test = CP_Oo[ii].split()
        map_object = map(float, test)
        temp = list(map_object)
        for jj in range(0,size,1):
            CP_O.append(temp[jj])

    CP_O = np.reshape(CP_O,(size,size))

    #import surface vibrations
    with open('vib_Basal.txt') as csv_file:
        data_base = csv.reader(csv_file, delimiter=' ')
        data_base = list(data_base)
        data_base = np.array(data_base).astype(float)

    data_base = data_base/1000

    with open(vibsfile) as csv_file:
        data = csv.reader(csv_file, delimiter='	')
        data = list(data)
        data = np.array(data).astype(float)

    data = data/1000

    #calculate Basal reference Gibbs
    qvib_C = np.array([])
    edge1 = len(data_base)-vib_remove
    zpe_C = 0
    for ii in range(0,edge1,1):
        zpe_C = zpe_C + data_base[ii]
        hold = np.exp(-data_base[ii]/kB/T)
        hold2 = 1/(1-hold)
        qvib_C = np.append(qvib_C, hold2)
    zpe_C = zpe_C/2
    qvib_C = np.prod(qvib_C)
    Gbase = zpe_C-kB*T*np.log(qvib_C)
    CP_C = Gbase/72

    #calculate surface Gibbs
    qvib = np.array([])
    edge2 = len(data)-vib_remove
    zpe = 0
    for ii in range(0,edge2,1):
        zpe = zpe + data[ii]
        hold3 = np.exp(-data[ii]/kB/T)
        hold4 = 1/(1-hold3)
        qvib = np.append(qvib, hold4)
    zpe = zpe/2
    qvib = np.prod(qvib)
    Gsurf = zpe-kB*T*np.log(qvib)

    #calculate pressure dependence
    Gtotf = np.array([])
    for ii in range(0,size,1):
        for jj in range(0,size,1):
            holdO = CP_O[ii,jj]
            holdH = CP_H[ii,jj]
            Gtot = Eform+Gsurf-(72-N_vac)*CP_C-N_O*holdO-N_H*holdH
            Gtotf = np.append(Gtotf, Gtot)

    Gtotf = np.reshape(Gtotf,(size,size))
    np.savetxt(outfiles[counter],Gtotf,fmt='%s')
    counter = counter+1